## Utilidades do Scikit-Learn
O Scikit-Learn é um dos pacotes mais conhecidos do mundo de Ciência de Dados por trazer diversas funcionalidades práticas para o dia a dia de qualquer cientista. Além dos modelos já implementados, como Árvores de Decisão, KNN, e Regressão Logística, trás também praticidades para pré-processamento de dados, que, se bem utilizados, mitigarão riscos de data leakage (vazamento de dados), além de trazer ganhos produtivos.

    Vazamento de dados no contexto de Machine Learning se refere ao erro do Cientista de Dados de acidentalmente compartilhar informações entre os conjuntos de teste e treino, causando uma avaliação possivelmente incorreta no desempenho do modelo preditivo.

2. Considerações Gerais
Muitos transformadores precisam de alguns parâmetros para definir como serão suas transformações. Porém, tais parâmetros devem ser ajustados observando apenas o conjunto de treino, exatamente para evitar o data leakage. Dessa forma, é importante separar bem qual é a base de treino e qual é a base teste. Com isso definido, instanciamos um objeto da classe do transformador e ajustamos os parâmetros usando o médoto fit(). Alguns transformadores exigem a variável resposta, outros não. Com os parâmetros encontrados, aplicamos o método transform() para efetivamente fazer a transformação. Dessa forma:

In [1]:
transformador = Transformador()
transformador.fit(x_treino, y_treino) 
x_treino_transformado = transformador.transform(x_treino)
x_teste_transformado = transformador.transform(x_teste)

10


Existe ainda o método fit_transform(), que basicamente chama o fit() e logo depois o trasnform(), retornando a tabela processada.

In [ ]:
transformador = Transformador()
x_treino_transformado = transformador.fit_transform(x_treino, y_treino) 
x_teste_transformado = transformador.transform(x_teste)

Existem diversos transformadores, cada um com suas particularidades e usos específicos e podem ser acessados na seção de Transformação do User Guide do sklearn. A parte de pipelines será abordada em outro momento desse módulo, enquanto que a extração de atributos será tema de outro módulo.


3. Binarizer

Muitas vezes, uma variável contínua precisa ser categorizada por não ter uma boa distribuição dos seus valores, ou ainda por um conhecimento especialista do cenário no qual ela está inserida. Por exemplo, se estamos tratando de seguros de automóveis, podemos categorizar as idades dos contratantes em <= 25 e > 25. Com o sklearn temos a classe Binarizer que dado um determinado limiar (parâmetro threshold, que por default é igual a 0), transforma uma variável contínua em binária.

In [ ]:
from sklearn.preprocessing import Binarizer

binarizer = Binarizer(threshold = 25)
x_treino.loc[:, ['idade']] = binarizer.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste.loc[:, ['idade']] = binarizer.transform(x_teste.loc[:, ['idade']]) 

4. KBinsDiscretizer

Em algumas situações, ao transformar uma variável contínua em binária perdemos muita informação a ponto de prejudicar muito a modelagem. Podemos então utilizar o KBinsDiscretizer para realizar uma discretização não tão severa quando o Binarizer. Como parâmetros, podemos usar

    n_bins: quantidade de valores discretos a ser criado;
    encode: formato de saída, que pode ser estilo one hot ou ordinal em uma coluna só;
    strategy: estratégia a ser usada para definir os limites, podendo ser uniforme, quantílico, ou até mesmo usar um K-Means.


In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

kbins = KBinsDiscretizer(n_bins = 4, enconde = 'ordinal', strategy = 'quantile')
x_treino.loc[:, ['idade']] = kbins.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste.loc[:, ['idade']] = kbins.transform(x_teste.loc[:, ['idade']]) 

5. MinMaxScaler

Muitos modelos são sensíveis a escalas diferentes muito altas das variáveis explicativas, como o KNN. Nesses casos, podemos fazer um reescalamento dos dados, deixando todas as features no intervalo de, por exemplo, 0 e 1, utilizando o MinMaxScaler. Casos outros intervalos sejam desejados, alteramos o parâmetro feature_range, passando uma tupla com o mínimo e o máximo.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler(feature_range = (0, 1))
x_treino.loc[:, ['idade']] = minmax.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste.loc[:, ['idade']] = minmax.transform(x_teste.loc[:, ['idade']]) 

6. StandardScaler

Com as mesmas motivações de reescalamento do MinMaxScaler, porém padronizando 
a distribuição da variável para $x=0$ e $\sigma=1$ após a transformação. 

Cada elemento transformado é dado pela equação:

$ x'= \frac{x-\bar{x}} {\sigma} $

Utilizando o sklearn:

In [ ]:
from sklearn.preprocessing import StandardScaler

stdscaler = StandardScaler()
x_treino.loc[:, ['idade']] = stdscaler.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste.loc[:, ['idade']] = stdscaler.transform(x_teste.loc[:, ['idade']]) 

7. OneHotEncoder

Em algum momentos, pode ser necessário levar as variáveis categóricas de um domínio ordinal, usando apenas uma coluna, para uma representação clara de não-ordenação, no estilo de dummies, onde cada coluna é uma flag representando se uma determinada instância é de uma determinada categoria. É similar ao pd.get_dummies(), porém com suporte aos métodos fit() e transform().

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
x_treino_ohe = ohe.fit_transform(x_treino.loc[:, ['cor']]) 
x_teste_ohe = ohe.transform(x_teste.loc[:, ['cor']]) 

x_treino = pd.concat([x_treino, pd.DataFrame(x_treino_ohe)], axis = 1)
x_teste = pd.concat([x_teste, pd.DataFrame(x_teste_ohe)], axis = 1)

8. OrdinalEncoder

Transforma uma variável categórica num formato numérico ordinal, detro do intervalo [0,ni−1]
, onde ni é a quantidade de categorias para cada atributo.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
x_treino.loc[:, ['idade']] = oe.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste.loc[:, ['idade']] = oe.transform(x_teste.loc[:, ['idade']]) 

9. SimpleImputer

Muitos modelos não trabalham bem com valores missing. Nesses casos, é importante utilizar de alguma estratégia de imputação de valores. O SimpleImputer possui diversas facilidades nesse sentido graças à sua versatilidade trazidas pelos seus parâmetros:
- ```missing_values```: o indicador de missing, que por padrão é np.nan, mas pode ser ajustado para -999 ou '', por exemplo;
- ```strategy```: qual a estratégia de imputação a ser utilizada, podendo incluir a média ('mean'), mediana ('median') e moda (most_frequent), todos desconsiderando missing em seus cálculos, ou ainda um valor constante (constant), a ser definido pelo parâmetro fill_value;
- ```fill_value```: valor utilizado caso seja strategy esteja definido como 'constant'.
- ```add_indicator```: muitas vezes, ser missing é uma ótima feature, e caso esse parâmetro seja setado como True, o transformador criará uma nova coluna para cada feature que foi realizada a imputação, indicando onde originalmente era missing.


In [ ]:
from sklearn.preprocessing import SimpleImputer

si = SimpleImputer(missing_values = -999, strategy = 'median', add_indicator = True)
x_treino_si = si.fit_transform(x_treino.loc[:, ['idade']]) 
x_teste_si = si.transform(x_teste.loc[:, ['idade']]) 

x_treino = pd.concat([x_treino, pd.DataFrame(x_treino_si)], axis = 1)
x_teste = pd.concat([x_teste, pd.DataFrame(x_teste_si)], axis = 1)